# シェル

## 通常の実行

In [2]:
use std::process::{Command,Stdio};
Command::new("ls")
		.arg("-l").arg("-a") // 或いは .args(&["-l","-a"])
		.spawn()
		.expect("実行に失敗しました");

## 実行方法をカスタマイズ
Rustでは上のように関数を繋げていくことでオプションを指定できる

### 実行内容の指定
* `Command::new(S)` : コマンドを指定
* `.arg(S)` : 引数を1つ追加
* `.args(&[S])` : 引数をまとめて追加

### IO
* `.stdin(Stdio)`,`.stdout(Stdio)`,`.stderr(Stdio)` : 入出力を指定  
	これらの値が指定できる
	* `Stdio::piped()`  
		別のプロセスへパイプしたり,文字列として渡す/受け取る
	* `Stdio::inherit()`  
		親プロセスの入出力を使用
	* `Stdio::null()`  
		出力を破棄する/空の入力
	* `Stdio::from(File)`  
		開いているファイルを入出力先として指定
	* `Stdio::from(ChildStdin)`, `Stdio::from(ChildStdout)`, `Stdio::from(ChildStderr)`  
		別プロセスの入出力をパイプする

### 環境変数
* `.env(K,V)` : 環境変数を指定する
* `.envs(&[(K,V)])` : 環境変数をまとめて指定する
* `.env_remove(K)` : 環境変数 `K` を削除する
* `.env_clear()` : 規定の環境変数を全て削除する

### カレントディレクトリ
* `.current_dir(S)` : プロセス内のカレントディレクトリを変更する

### 実行

* `.status()` → `Result<ExitStatus>` : 実行を開始し,実行が終わるのを待って,ステータスを返す  
	これを使用した場合,標準では stdin,stdout,stderr のいづれもインターセプトされない (`Stdio::inheirt()` が指定される)
	* `ExitStatus.success()` → `bool` : 成功したか否か
	* `ExitStatus.code()` → `Option<i32>` : 終了コード  
		シグナルで中断した場合は `None` になる

* `.output()` → `Result<Output>` : 実行を開始し,実行が終わるのを待って,結果を返す  
	これを使用した場合,標準では stdout,stderr 共にインターセプトされる (`Stdio::piped()` が指定される)  
	逆に, `Stdio::piped()` が指定されていないIOはアクセスできない
	* `Output.stdout` → `Vec<u8>` : stdoutの内容
	* `Output.stderr` → `Vec<u8>` : stderrの内容
	* `Output.status` → `ExitStatus` : ステータス  
		`ExitStatus` の内容は `.status()`　の項を参照

* `.spawn()` → `Result<Child>` : 実行を開始し,子プロセスを返す  
	これを使用した場合,標準では stdin,stdout,stderr のいづれもインターセプトされない (`Stdio::inheirt()` が指定される)  
	`.spawn()` の時点では実行は完了していないので, `Child.wait()` を使用する
	* `Child.id()` → `u32` : プロセスID
	* `Child.kill()` : 強制終了する
	* `Child.stdin` → `Option<ChildStdin>` : stdoinの内容 (`Stdio::piped()` が指定されている場合に有効)  
		別プロセスへパイプするには, `Stdio::from(ChildStdin)` にこれを渡せばよい
		* `ChildStdin.write(&[u8])` : stdinに書き込む
	* `Child.stdout`,`Child.stderr` → `Option<ChildStdout>`,`Option<ChildStderr>` : stdout,stderrの内容 (`Stdio::piped()` が指定されている場合に有効)  
		別プロセスへパイプするには, `Stdio::from(ChildStdout/err)` にこれを渡せばよい
		* `ChildStdout/err.read(&mut [u8])` : stdout,stderrから読み込む
		* `ChildStdout/err.read_to_vec(&mut Vec<u8>)` : stdout,stderrから読み込む
		* `ChildStdout/err.read_to_string(&mut String)` : stdout,stderrから読み込む
	* `Child.wait()` → `ExitStatus` : 終了を待って,ステータスを返す